In [8]:
import requests
from bs4 import BeautifulSoup
import time
import urllib3

# Suprimir avisos de SSL inseguros
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# URL base do site
base_url = "https://www.ogol.com.br"

# Cabeçalhos para a requisição
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:129.0) Gecko/20100101 Firefox/129.0'
}

# Função para fazer uma requisição com tentativa e atraso
def make_request(url, headers, max_retries=5, delay=10):
    for i in range(max_retries):
        try:
            response = requests.get(url, headers=headers, verify=False)  # Ignorar verificação SSL
            if response.status_code == 200:
                return response
            elif response.status_code == 429:
                print(f"Recebido status code 429. Tentativa {i+1} de {max_retries}. Aguardando {delay} segundos...")
                time.sleep(delay)  # Espera antes de tentar novamente
            else:
                print(f"Falha ao acessar o site. Status code: {response.status_code}")
                return None
        except requests.exceptions.SSLError as e:
            print(f"Erro SSL: {e}. Tentativa {i+1} de {max_retries}. Aguardando {delay} segundos...")
            time.sleep(delay)  # Espera antes de tentar novamente
    return None

# Lista para armazenar os links de campeonatos
campeonatos = []

# Pegar os links dos campeonatos da página principal
response = make_request(base_url, headers)
if response:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Buscando os links dos campeonatos conforme a estrutura da primeira imagem
    text_divs = soup.find_all('div', class_='text')
    
    for text_div in text_divs:
        a_tag = text_div.find('a')
        if a_tag and a_tag.get('href'):
            link = a_tag['href']
            # Adicionar somente links que sejam de campeonatos
            if '/competicao/' in link:
                campeonatos.append(base_url + link)

# Lista para armazenar os links dos clubes
clubes = []

# Iterar sobre cada link de campeonato
for campeonato_url in campeonatos:
    
    # Fazer a requisição à página do campeonato
    response = make_request(campeonato_url, headers)
    
    if response:
        # Parsear o conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Buscar links de clubes dentro de <div class="text">
        text_divs = soup.find_all('div', class_='text')
        
        for text_div in text_divs:
            a_tag = text_div.find('a')
            if a_tag and a_tag.get('href'):
                link = a_tag['href']
                # Adicionar links de clubes na lista de clubes
                if '/equipe/' in link:
                    clubes.append(base_url + link)
        
        # Buscar links de clubes dentro das tabelas de classificação
        rows = soup.select('div#fixture_games table tbody tr td.text a')
        
        for row in rows:
            if row.get('href'):
                link = row.get('href')
                # Adicionar links de clubes na lista de clubes
                if '/equipe/' in link:
                    clubes.append(base_url + link)

# Remover duplicatas na lista de clubes
clubes = list(set(clubes))

# Exibir a lista de links de campeonatos encontrados
print("Links de Campeonatos:", campeonatos)

# Exibir a lista de links de clubes encontrados
print("Links de Clubes:", clubes)

# Nova seção: Coletar links de atletas e equipe técnica

# Lista para armazenar os links de atletas e equipe técnica
atletas_equipe_links = []

# Iterar sobre cada link de clube para pegar os links de atletas e equipe técnica
for clube_url in clubes:
    
    # Fazer a requisição à página do clube
    response = make_request(clube_url, headers)
    
    if response:
        # Parsear o conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # **Primeira imagem**: Encontrar o <div> com id 'team_squad' e buscar os links dentro de <a> em href
        team_squad_div = soup.find('div', id='team_squad')
        
        if team_squad_div:
            text_divs = team_squad_div.find_all('div', class_='text')
            for text_div in text_divs:
                a_tag = text_div.find('a')
                if a_tag and a_tag.get('href'):
                    link = a_tag['href']
                    atletas_equipe_links.append(base_url + link)
        
        # **Segunda imagem**: Encontrar o <div> com id 'team_staff' e buscar os links dentro de <a> em href
        team_staff_div = soup.find('div', id='team_staff')
        
        if team_staff_div:
            text_divs = team_staff_div.find_all('div', class_='text')
            for text_div in text_divs:
                a_tag = text_div.find('a')
                if a_tag and a_tag.get('href'):
                    link = a_tag['href']
                    atletas_equipe_links.append(base_url + link)

# Remover duplicatas na lista de atletas e equipe técnica
atletas_equipe_links = list(set(atletas_equipe_links))

# Exibir a lista de links de atletas e equipe técnica
print("Links de Atletas e Equipe Técnica:", atletas_equipe_links)

Links de Campeonatos: ['https://www.ogol.com.br/competicao/campeonato-brasileiro', 'https://www.ogol.com.br/competicao/copa-do-brasil', 'https://www.ogol.com.br/competicao/supercopa-do-brasil-849', 'https://www.ogol.com.br/competicao/brasileiro-serie-b', 'https://www.ogol.com.br/competicao/brasileiro-serie-c', 'https://www.ogol.com.br/competicao/brasileiro-serie-d', 'https://www.ogol.com.br/competicao/liga-inglesa', 'https://www.ogol.com.br/competicao/liga-espanhola', 'https://www.ogol.com.br/competicao/campeonato-italiano', 'https://www.ogol.com.br/competicao/liga-portuguesa', 'https://www.ogol.com.br/competicao/bundesliga', 'https://www.ogol.com.br/competicao/campeonato-frances', 'https://www.ogol.com.br/competicao/copa-sul-americana', 'https://www.ogol.com.br/competicao/recopa-sul-americana', 'https://www.ogol.com.br/competicao/copa-libertadores', 'https://www.ogol.com.br/competicao/supercopa-europeia', 'https://www.ogol.com.br/competicao/liga-dos-campeoes', 'https://www.ogol.com.br

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Links de Atletas e Equipe Técnica: ['https://www.ogol.com.br/jogador/juan-diaz/784169?epoca_id=153', 'https://www.ogol.com.br/jogador/ismael-diaz/375161?epoca_id=153', 'https://www.ogol.com.br/jogador/tomasz-kedziora/251780?edicao_id=168931', 'https://www.ogol.com.br/jogador/agustin-ruberto/1179453?epoca_id=153', 'https://www.ogol.com.br/jogador/aldo-maiz-/626723?epoca_id=153', 'https://www.ogol.com.br/jogador/adu-ares/932250?epoca_id=154', 'https://www.ogol.com.br/jogador/nicolas/166020?epoca_id=153', 'https://www.ogol.com.br/jogador/jo-/5171?epoca_id=153', 'https://www.ogol.com.br/jogador/karol-mets/184361?epoca_id=154', 'https://www.ogol.com.br/jogador/martha/474419?epoca_id=153', 'https://www.ogol.com.br/jogador/mateus-canhota/510852?epoca_id=153', 'https://www.ogol.com.br/jogador/andrew-ventura/641489?epoca_id=154', 'https://www.ogol.com.br/jogador/mario-gjata/842895?epoca_id=154', 'https://www.ogol.com.br/jogador/nikola-serafimov/738227?epoca_id=154', 'https://www.ogol.com.br/jog

In [9]:
atletas_equipe_links = list(set(atletas_equipe_links))

In [10]:
len(atletas_equipe_links)

20033

In [21]:
import requests
from bs4 import BeautifulSoup
import time
import urllib3

# Suprimir avisos de SSL inseguros
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# URL base do site
base_url = "https://www.ogol.com.br"

# Cabeçalhos para a requisição
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:129.0) Gecko/20100101 Firefox/129.0'
}

# Função para fazer uma requisição com tentativa e atraso
def make_request(url, headers, max_retries=5, delay=10):
    for i in range(max_retries):
        try:
            response = requests.get(url, headers=headers, verify=False)  # Ignorar verificação SSL
            if response.status_code == 200:
                return response
            elif response.status_code == 429:
                print(f"Recebido status code 429. Tentativa {i+1} de {max_retries}. Aguardando {delay} segundos...")
                time.sleep(delay)  # Espera antes de tentar novamente
            else:
                print(f"Falha ao acessar o site. Status code: {response.status_code}")
                return None
        except requests.exceptions.SSLError as e:
            print(f"Erro SSL: {e}. Tentativa {i+1} de {max_retries}. Aguardando {delay} segundos...")
            time.sleep(delay)  # Espera antes de tentar novamente
    return None

# Lista para armazenar os links de campeonatos
campeonatos = []

# Pegar os links dos campeonatos da página principal
response = make_request(base_url, headers)
if response:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Buscando os links dos campeonatos conforme a estrutura da primeira imagem
    text_divs = soup.find_all('div', class_='text')
    
    for text_div in text_divs:
        a_tag = text_div.find('a')
        if a_tag and a_tag.get('href'):
            link = a_tag['href']
            # Adicionar somente links que sejam de campeonatos
            if '/competicao/' in link:
                campeonatos.append(base_url + link)

# Lista para armazenar os links dos clubes
clubes = []

# Iterar sobre cada link de campeonato
for campeonato_url in campeonatos:
    
    # Fazer a requisição à página do campeonato
    response = make_request(campeonato_url, headers)
    
    if response:
        # Parsear o conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Buscar links de clubes dentro de <div class="text">
        text_divs = soup.find_all('div', class_='text')
        
        for text_div in text_divs:
            a_tag = text_div.find('a')
            if a_tag and a_tag.get('href'):
                link = a_tag['href']
                # Adicionar links de clubes na lista de clubes
                if '/equipe/' in link:
                    clubes.append(base_url + link)
        
        # Buscar links de clubes dentro das tabelas de classificação
        rows = soup.select('div#fixture_games table tbody tr td.text a')
        
        for row in rows:
            if row.get('href'):
                link = row.get('href')
                # Adicionar links de clubes na lista de clubes
                if '/equipe/' in link:
                    clubes.append(base_url + link)

# Remover duplicatas na lista de clubes
clubes = list(set(clubes))

# Exibir a lista de links de campeonatos encontrados
print("Links de Campeonatos:", campeonatos)

# Exibir a lista de links de clubes encontrados
print("Links de Clubes:", clubes)

# Nova seção: Coletar links de atletas e equipe técnica

# Lista para armazenar os links de atletas e equipe técnica
atletas_equipe_links = []

# Iterar sobre cada link de clube para pegar os links de atletas e equipe técnica
for clube_url in clubes:
    
    # Fazer a requisição à página do clube
    response = make_request(clube_url, headers)
    
    if response:
        # Parsear o conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # **Primeira imagem**: Encontrar o <div> com id 'team_squad' e buscar os links dentro de <a> em href
        team_squad_div = soup.find('div', id='team_squad')
        
        if team_squad_div:
            text_divs = team_squad_div.find_all('div', class_='text')
            for text_div in text_divs:
                a_tag = text_div.find('a')
                if a_tag and a_tag.get('href'):
                    link = a_tag['href']
                    atletas_equipe_links.append(base_url + link)
        
        # **Segunda imagem**: Encontrar o <div> com id 'team_staff' e buscar os links dentro de <a> em href
        team_staff_div = soup.find('div', id='team_staff')
        
        if team_staff_div:
            text_divs = team_staff_div.find_all('div', class_='text')
            for text_div in text_divs:
                a_tag = text_div.find('a')
                if a_tag and a_tag.get('href'):
                    link = a_tag['href']
                    atletas_equipe_links.append(base_url + link)

# Remover duplicatas na lista de atletas e equipe técnica
atletas_equipe_links = list(set(atletas_equipe_links))

# Exibir a lista de links de atletas e equipe técnica
print("Links de Atletas e Equipe Técnica:", atletas_equipe_links)

# Nova seção: Coletar informações detalhadas dos atletas e equipe técnica

# Listas para armazenar as informações extraídas
nomes = []
data_nascimento = []
idade = []
nacionalidades = []  # Corrigido para evitar conflitos de variável

# Iterar sobre cada link de atleta/equipe para extrair informações
for atleta_url in atletas_equipe_links:
    
    # Fazer a requisição à página do atleta/equipe
    response = make_request(atleta_url, headers)
    
    if response:
        # Parsear o conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extrair o nome completo
        nome_div = soup.find('div', class_='bio')
        if nome_div:
            nome = nome_div.get_text(strip=True)
        else:
            nome = 'sem informação'
        nomes.append(nome)
        
        # Extrair a data de nascimento
        data_nasc_div = soup.find('div', class_='bio_half')
        if data_nasc_div:
            data_nasc = data_nasc_div.get_text(strip=True)
        else:
            data_nasc = 'sem informação'
        data_nascimento.append(data_nasc)
        
        # Extrair a idade (removendo os parênteses)
        idade_span = soup.find('span', class_='small')
        if idade_span:
            idade_str = idade_span.get_text(strip=True).replace('(', '').replace(')', '')
        else:
            idade_str = 'sem informação'
        idade.append(idade_str)
        
        # Extrair a nacionalidade
        nacionalidade_div = soup.find('div', class_='text')
        if nacionalidade_div:
            nacionalidade = nacionalidade_div.get_text(strip=True)
        else:
            nacionalidade = 'sem informação'
        nacionalidades.append(nacionalidade)

# Exibir as listas extraídas
print("Nomes:", nomes)
print("Data de Nascimento:", data_nascimento)
print("Idade:", idade)
print("Nacionalidade:", nacionalidades)

Links de Campeonatos: ['https://www.ogol.com.br/competicao/campeonato-brasileiro', 'https://www.ogol.com.br/competicao/copa-do-brasil', 'https://www.ogol.com.br/competicao/supercopa-do-brasil-849', 'https://www.ogol.com.br/competicao/brasileiro-serie-b', 'https://www.ogol.com.br/competicao/brasileiro-serie-c', 'https://www.ogol.com.br/competicao/brasileiro-serie-d', 'https://www.ogol.com.br/competicao/liga-inglesa', 'https://www.ogol.com.br/competicao/liga-espanhola', 'https://www.ogol.com.br/competicao/campeonato-italiano', 'https://www.ogol.com.br/competicao/liga-portuguesa', 'https://www.ogol.com.br/competicao/bundesliga', 'https://www.ogol.com.br/competicao/campeonato-frances', 'https://www.ogol.com.br/competicao/copa-sul-americana', 'https://www.ogol.com.br/competicao/recopa-sul-americana', 'https://www.ogol.com.br/competicao/copa-libertadores', 'https://www.ogol.com.br/competicao/supercopa-europeia', 'https://www.ogol.com.br/competicao/liga-dos-campeoes', 'https://www.ogol.com.br

In [20]:
import requests
from bs4 import BeautifulSoup
import time
import urllib3

# Suprimir avisos de SSL inseguros
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# URL base do site
base_url = "https://www.ogol.com.br"

# Cabeçalhos para a requisição
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:129.0) Gecko/20100101 Firefox/129.0'
}

# Função para fazer uma requisição com tentativa e atraso
def make_request(url, headers, max_retries=5, delay=10):
    for i in range(max_retries):
        try:
            response = requests.get(url, headers=headers, verify=False)  # Ignorar verificação SSL
            if response.status_code == 200:
                print(f"Acesso bem-sucedido a: {url}")
                return response
            elif response.status_code == 429:
                print(f"Recebido status code 429. Tentativa {i+1} de {max_retries}. Aguardando {delay} segundos...")
                time.sleep(delay)  # Espera antes de tentar novamente
            else:
                print(f"Falha ao acessar o site. Status code: {response.status_code}")
                return None
        except requests.exceptions.SSLError as e:
            print(f"Erro SSL: {e}. Tentativa {i+1} de {max_retries}. Aguardando {delay} segundos...")
            time.sleep(delay)  # Espera antes de tentar novamente
    return None

# Lista para armazenar os links de campeonatos
campeonatos = []

# Pegar os links dos campeonatos da página principal
response = make_request(base_url, headers)
if response:
    soup = BeautifulSoup(response.content, 'html.parser')
    print(f"Título da página principal: {soup.title.string}")
    
    # Buscando os links dos campeonatos conforme a estrutura da primeira imagem
    text_divs = soup.find_all('div', class_='text')
    print(f"Número de divs encontradas com classe 'text': {len(text_divs)}")
    
    for text_div in text_divs:
        a_tag = text_div.find('a')
        if a_tag and a_tag.get('href'):
            link = a_tag['href']
            # Adicionar somente links que sejam de campeonatos
            if '/competicao/' in link:
                campeonatos.append(base_url + link)

# Verificar se campeonatos foram encontrados
print(f"Links de Campeonatos encontrados: {campeonatos}")

# Lista para armazenar os links dos clubes
clubes = []

# Iterar sobre cada link de campeonato
for campeonato_url in campeonatos:
    
    # Fazer a requisição à página do campeonato
    response = make_request(campeonato_url, headers)
    
    if response:
        # Parsear o conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        print(f"Título da página do campeonato: {soup.title.string}")
        
        # Buscar links de clubes dentro de <div class="text">
        text_divs = soup.find_all('div', class_='text')
        print(f"Número de divs encontradas com classe 'text' na página do campeonato: {len(text_divs)}")
        
        for text_div in text_divs:
            a_tag = text_div.find('a')
            if a_tag and a_tag.get('href'):
                link = a_tag['href']
                # Adicionar links de clubes na lista de clubes
                if '/equipe/' in link:
                    clubes.append(base_url + link)
        
        # Buscar links de clubes dentro das tabelas de classificação
        rows = soup.select('div#fixture_games table tbody tr td.text a')
        print(f"Número de links encontrados na tabela de classificação: {len(rows)}")
        
        for row in rows:
            if row.get('href'):
                link = row.get('href')
                # Adicionar links de clubes na lista de clubes
                if '/equipe/' in link:
                    clubes.append(base_url + link)

# Remover duplicatas na lista de clubes
clubes = list(set(clubes))

# Verificar se clubes foram encontrados
print(f"Links de Clubes encontrados: {clubes}")

# Nova seção: Coletar links de atletas e equipe técnica

# Lista para armazenar os links de atletas e equipe técnica
atletas_equipe_links = []

# Iterar sobre cada link de clube para pegar os links de atletas e equipe técnica
for clube_url in clubes:
    
    # Fazer a requisição à página do clube
    response = make_request(clube_url, headers)
    
    if response:
        # Parsear o conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        print(f"Título da página do clube: {soup.title.string}")
        
        # **Primeira imagem**: Encontrar o <div> com id 'team_squad' e buscar os links dentro de <a> em href
        team_squad_div = soup.find('div', id='team_squad')
        if team_squad_div:
            text_divs = team_squad_div.find_all('div', class_='text')
            print(f"Número de divs encontradas com classe 'text' na página do clube (team_squad): {len(text_divs)}")
            for text_div in text_divs:
                a_tag = text_div.find('a')
                if a_tag and a_tag.get('href'):
                    link = a_tag['href']
                    atletas_equipe_links.append(base_url + link)
        
        # **Segunda imagem**: Encontrar o <div> com id 'team_staff' e buscar os links dentro de <a> em href
        team_staff_div = soup.find('div', id='team_staff')
        if team_staff_div:
            text_divs = team_staff_div.find_all('div', class_='text')
            print(f"Número de divs encontradas com classe 'text' na página do clube (team_staff): {len(text_divs)}")
            for text_div in text_divs:
                a_tag = text_div.find('a')
                if a_tag and a_tag.get('href'):
                    link = a_tag['href']
                    atletas_equipe_links.append(base_url + link)

# Remover duplicatas na lista de atletas e equipe técnica
atletas_equipe_links = list(set(atletas_equipe_links))

# Limitar a iteração aos dois primeiros links de atletas/equipe para testes
atletas_equipe_links_teste = atletas_equipe_links[:2]

# Verificar se links de atletas e equipe técnica foram encontrados
print(f"Links de Atletas e Equipe Técnica para teste: {atletas_equipe_links_teste}")

# Nova seção: Coletar informações detalhadas dos atletas e equipe técnica usando os dois primeiros links

# Listas para armazenar as informações extraídas
nomes = []
data_nascimento = []
idade = []
nacionalidades = []

# Iterar sobre cada link de atleta/equipe para extrair informações
for atleta_url in atletas_equipe_links_teste:
    
    # Fazer a requisição à página do atleta/equipe
    response = make_request(atleta_url, headers)
    
    if response:
        # Parsear o conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        print(f"Título da página do atleta/equipe: {soup.title.string}")
        
        # Extrair o nome completo
        nome_div = soup.find('div', class_='bio')
        if nome_div:
            nome = nome_div.get_text(strip=True)
        else:
            nome = 'sem informação'
        nomes.append(nome)
        
        # Extrair a data de nascimento
        data_nasc_div = soup.find('div', class_='bio_half')
        if data_nasc_div:
            data_nasc = data_nasc_div.get_text(strip=True)
        else:
            data_nasc = 'sem informação'
        data_nascimento.append(data_nasc)
        
        # Extrair a idade (removendo os parênteses)
        idade_span = soup.find('span', class_='small')
        if idade_span:
            idade_str = idade_span.get_text(strip=True).replace('(', '').replace(')', '')
        else:
            idade_str = 'sem informação'
        idade.append(idade_str)
        
        # Extrair a nacionalidade
        nacionalidade_div = soup.find('div', class_='text')
        if nacionalidade_div:
            nacionalidade = nacionalidade_div.get_text(strip=True)
        else:
            nacionalidade = 'sem informação'
        nacionalidades.append(nacionalidade)

# Exibir as listas extraídas
print("Nomes:", nomes)
print("Data de Nascimento:", data_nascimento)
print("Idade:", idade)
print("Nacionalidade:", nacionalidades)

Acesso bem-sucedido a: https://www.ogol.com.br
Título da página principal: ogol.com.br :: Tudo sobre futebol
Número de divs encontradas com classe 'text': 170
Links de Campeonatos encontrados: ['https://www.ogol.com.br/competicao/campeonato-brasileiro', 'https://www.ogol.com.br/competicao/copa-do-brasil', 'https://www.ogol.com.br/competicao/supercopa-do-brasil-849', 'https://www.ogol.com.br/competicao/brasileiro-serie-b', 'https://www.ogol.com.br/competicao/brasileiro-serie-c', 'https://www.ogol.com.br/competicao/brasileiro-serie-d', 'https://www.ogol.com.br/competicao/liga-inglesa', 'https://www.ogol.com.br/competicao/liga-espanhola', 'https://www.ogol.com.br/competicao/campeonato-italiano', 'https://www.ogol.com.br/competicao/liga-portuguesa', 'https://www.ogol.com.br/competicao/bundesliga', 'https://www.ogol.com.br/competicao/campeonato-frances', 'https://www.ogol.com.br/competicao/copa-sul-americana', 'https://www.ogol.com.br/competicao/recopa-sul-americana', 'https://www.ogol.com.

In [17]:
len(campeonatos)

31